In [13]:
import nltk
import numpy as np
import pickle
import json
import random
from keras.models import load_model
from nltk.stem import WordNetLemmatizer

In [14]:
lemmatizer = WordNetLemmatizer()

In [20]:
# chat initialization
model = load_model("chatbot_model_latest.h5")
intents = json.loads(open("custom-intents.json").read())
data_file = open("custom-intents.json").read()
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [21]:
# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [22]:
intent = predict_class("hi", model)

1/1 [==============================] - 0s 97ms/step


In [23]:
res = getResponse(intent, json.loads(data_file))

In [24]:
res

'Greetings! How can I be of service?'

In [25]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [27]:
while True:
    message = input("")
    if message == "quit":
        break
    print(chatbot_response(message))

1/1 [==============================] - 0s 47ms/step
I was developed by Abdulmunim Jundurahman as part of a side open-source project for Addis Ababa Institute of Technology.
1/1 [==============================] - 0s 48ms/step
Hello, {n}! What can I do for you?
1/1 [==============================] - 0s 33ms/step
I'm sorry, I couldn't understand your question. I am currently under heavy development and may not have answers to all questions. It will be improved over time as more data becomes available.
1/1 [==============================] - 0s 36ms/step
I'm sorry, I couldn't understand your question. I am currently under heavy development and may not have answers to all questions. It will be improved over time as more data becomes available.
1/1 [==============================] - 0s 34ms/step
I'm sorry, I couldn't understand your question. I am currently under heavy development and may not have answers to all questions. It will be improved over time as more data becomes available.
1/1 [===